### FastText 
- Word2Vec에서 OOV(사전에 없는 용어) 문제를 해결하기 위한 모델 
- Word2Vec에서는 '강아지' 와 '강아지들' 문구를 다른 단어로 생각
- FastText는 Word2Vec의 학습 방식은 비슷 
    - 단어를 한글씩 잘게 쪼개서 단어의 유사도를 생성 
    - '강아지' -> '강', '아', '지', '강아', '아지', '강아지'(n-gram 방식)
    - word2vec과 기본적인 매개변수는 같지만 min_n, max_n 매개변수가 존재 
    - subword의 최소 길이와 최대 길이를 설정 
    - min_n = 1, max_n =1 --> subword를 사용하지 않겠다 -> Word2Vec 같은 형태로 학습

In [1]:
from gensim.models import Word2Vec, FastText

In [3]:
# 샘플 문장 생성
sentences = [
    ['이커머스', '데이터', '분석', '진행'], 
    ['상품', '리뷰', '기반', '감성', '분석', '합니다'], 
    ['형태소', '단위', '임베딩', '가능']
]
# Fast Text 모델에 학습 
model  = FastText(
    sentences=sentences, 
    vector_size= 50,        # 단위 벡터의 차원 수
    window=3,               # 주변의 확인할 단어의 개수
    min_count=1,            # 최소 출현 횟수
    sg = 1,                 # Skip-gram 방식으로 확률 계산
    epochs= 10,             # 학습 반복 횟수
    min_n = 2, 
    max_n= 6
)

In [ ]:
# 특정 단어의 벡터를 확인 
model.wv['이커머스']

In [5]:
# 유사 단어를 확인 
model.wv.most_similar('데이터', topn=3)

[('이커머스', 0.17707844078540802),
 ('합니다', 0.17073467373847961),
 ('가능', 0.16921374201774597)]

In [ ]:
# 학습 내용에 없는 단어를 확인 
model.wv['감정']

In [7]:
# Word2vec을 이용해서 sentences 학습하고 없는 단어 확인 
model2 = Word2Vec(
    sentences=sentences, 
    window=3, 
    vector_size=50, 
    min_count=1, 
    sg = 1, 
    epochs=10
)

In [9]:
# 학습한 내용에 없는 단어를 출력 
# model2.wv['감정']
# Word2Vec은 사전에 없는 단어를 단위벡터로 확인하면 에러가 발생 

- Word2Vec과 FastText와 단어간의 유사도 차이 확인 

    - '강아지', '강아지들' 두개의 단어를 유사한 단어
        - Word2Vec은 다른 단어로 인식 -> 유사도 낮게 
        - FastText는 비슷한 단어로 인식 -> 유사도 높게 

In [10]:
sentences2 = [
    ["고양이", "고양이들", '귀엽다', '동물', '반려동물'], 
    ['강아지', '강아지들', '귀엽다', '동물', '반려동물'], 
    ['달리다', '달리는', '달림', '걷다', '걷는'], 
    ['빠르다', '빠른', '느리다', '느림'], 
    ['예쁘다', '예쁨', '예쁜', '매력적이다'], 
    ['컴퓨터', '컴퓨팅', '컴퓨터들', '기계']
]

# 모델학습 (word2vec, fasttext)
w2v = Word2Vec(
    sentences=sentences2, 
    vector_size=100,
    window = 3, 
    min_count=1, 
    sg =1, 
    epochs=50, 
    seed = 42
    )
ft = FastText(
    sentences=sentences2, 
    vector_size= 100,
    window = 3, 
    min_count=1, 
    sg = 1, 
    epochs=50, 
    seed=42, 
    min_n= 2, 
    max_n=6
)

In [11]:
# Word2Vec의 유사도 확인 
print(
    'Word2Vec 유사도', w2v.wv.similarity('강아지', '강아지들')
)
# FastText의 유사도 확인 
print(
    'FastText 유사도', ft.wv.similarity('강아지', '강아지들')
)

Word2Vec 유사도 0.20342888
FastText 유사도 0.385201


In [13]:
# 비교 대상 단어들 
test_text = [
    ['강아지', '강아지들'], 
    ['고양이', '고양이들'], 
    ['달리다', '달리는'], 
    ['예쁘다', '예쁜'], 
    ['컴퓨터', '컴퓨팅'], 
    ['빠르다', '느림']
]

for a, b in test_text:
    w2v_sim = float(w2v.wv.similarity(a, b))    # 유사도 데이터를 실수형 변경
    ft_sim = float(ft.wv.similarity(a, b))      # 유사도 데이터를 실수형 변경
    print(f"{a} - {b}의 유사도 : Word2Vec ({round(w2v_sim, 4)}) \
          FastText ({round(ft_sim, 4)})")


강아지 - 강아지들의 유사도 : Word2Vec (0.2034)           FastText (0.3852)
고양이 - 고양이들의 유사도 : Word2Vec (-0.1262)           FastText (0.3965)
달리다 - 달리는의 유사도 : Word2Vec (-0.0706)           FastText (0.2667)
예쁘다 - 예쁜의 유사도 : Word2Vec (0.1392)           FastText (0.0696)
컴퓨터 - 컴퓨팅의 유사도 : Word2Vec (-0.0516)           FastText (0.3647)
빠르다 - 느림의 유사도 : Word2Vec (0.0108)           FastText (-0.0127)


In [17]:
# 상품 명을 기준으로 특정 상품을 검색 시 연관 된 상품의 목록을 확인 

products = {
    'P001' : '무선 이어폰 블루투스 노이즈캔슬링 충전케이스', 
    'P002' : '유선 이어폰 하이파이 금도금 플러그', 
    'P003' : '게이밍 마우스 RGB 경량 디자인', 
    'P004' : '무선 마우스 초경량 블루투스 듀얼모드', 
    'P005' : '헤드폰 노이즈캔슬링 유선'
}
# products에서 FastText 통해 학습을 시키기 위해 데이터를 추출 
# 필요한 데이터는 dict형 데이터에서 values
datas = products.values()
# datas을 공백을 기준으로 나눠준다. 
tokens = []
for data in datas:
    tokens.append( data.split() )

tokens


[['무선', '이어폰', '블루투스', '노이즈캔슬링', '충전케이스'],
 ['유선', '이어폰', '하이파이', '금도금', '플러그'],
 ['게이밍', '마우스', 'RGB', '경량', '디자인'],
 ['무선', '마우스', '초경량', '블루투스', '듀얼모드'],
 ['헤드폰', '노이즈캔슬링', '유선']]

In [18]:
# 토큰화 된 데이터를 이용하여 FastText에 학습 
ft2 = FastText(
    sentences= tokens, 
    vector_size= 100, 
    window = 3, 
    min_count=1, 
    sg = 1, 
    epochs= 10, 
    min_n = 3, 
    max_n = 6, 
    seed = 42
)   # subword를 사용하는 FastText 모델에 학습 

In [ ]:
# 단위 벡터 확인  -> vector_size 차원의 좌표
ft2.wv['마우스']

In [23]:
import numpy as np
# 문장의 평균 벡터를 구하는 함수 정의 
def sent_vec(token, type = None):
    vecs = []
    for w in token:
        vecs.append(ft.wv[w])
    # 해당 vecs가 존재하지 않는다면 희소행렬을 되돌려준다. 
    if not vecs:
        return np.zeros(ft.vector_size)
    print(np.array(vecs).shape)
    v = np.mean(vecs, axis=0)
    # 일반적인 문장의 평균 백터를 구하는 식 
    # 성능을 올리기 위해서는 L2 정규화 -> 벡터의 거리로 나눠준다. 
    if type == 'l2':
        v = v / (np.linalg.norm(v) + 1e-12)     
        # 1e-12을 더한 이유는 0으로 나눠지는것을 방지 하기 위함
    print(v.shape)
    return v


In [ ]:
# 토큰화 된 데이터들을 평균 벡터로 변환 
item_vecs = []
for t in tokens :
    # print(sent_vec(t))
    # break
    item_vecs.append(sent_vec(t))

In [26]:
# 평균 벡터를 이용해서 코사인 유사도를 확인 하기 위해서 
# sklearn  코사인 유사도 함수를 로드 
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
idx = list(products.keys()).index('P003')
cosine_similarity(item_vecs[idx:idx+1], item_vecs)

cosine_similarity([item_vecs[idx]], item_vecs)

array([[0.08545259, 0.09662677, 1.0000001 , 0.21450484, 0.09070536]],
      dtype=float32)

In [37]:
# 코사인 유사도를 이용해서 가장 근접한 상품의 이름을 출력 
# 추천 하는 함수를 생성 
def recommand_by_text(product_id):
    # 해당 상품명의 위치 값 -> 
    # item_vecs위치를 이용하여 코사인 유사도를 생성하기 위함
    # 상품의 id값들은 products라는 dict에서 해당 id의 위치를 저장 
    idx = list(products.keys()).index(product_id)

    # item_vecs에서 해당 인덱스의 값과 전체 vectors의 값을 비교
    # 코사인 유사도를 확인하면 하나의 문장을 유사도를 확인했기 때문에 2차원이 아닌 
    # 1차원 데이터를 생성 
    sims = cosine_similarity(item_vecs[idx:idx+1], item_vecs).ravel()
    # 내림차순 정렬 
    order = sims.argsort()[::-1]
    # 추천 단어들을 출력 
    rec = []
    for i in order:
        # 코사인 유사도에서 같은 id 라면 추천하지 않는다. 
        if list(products.keys())[i] != product_id:
            # 상품의 id와 유사도의 값들을 rec 추가
            rec.append([ list(products.keys())[i], sims[i] ])
    return rec
    

In [39]:
rec_list = recommand_by_text('P001')

In [43]:
for idx, (pid, _) in enumerate(rec_list):
    # 유사도가 높은 상위 2개 상품의 이름을 확인 
    print(products[pid])
    if idx == 1:
        break

무선 마우스 초경량 블루투스 듀얼모드
유선 이어폰 하이파이 금도금 플러그
